In [2]:
%reload_ext autoreload
%autoreload 2

import os

os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
import pytz
import seaborn as sns
from datetime import datetime, timedelta
import concurrent.futures

from watttime import WattTimeForecast, WattTimeHistorical

import data.s3 as s3u
import evaluation.eval_framework as efu

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

s3 = s3u.s3_utils()
key = "20240726_1k_synth_users_163_days.csv"
generated_data = s3.load_csvdataframe(file=key)
generated_data = generated_data[-8000:]

In [5]:
generated_data

,index,distinct_dates,user_type,plug_in_time,unplug_time,initial_charge,total_seconds_to_95,full_charge_time,length_plugged_in,charged_kWh_actual,final_perc_charged,final_charge_time,uncharged,total_capacity,power_output_rate,total_intervals_plugged_in,charge_MWh_needed,charged_MWh_actual,MWh_fraction
155000,150,2024-06-10,r5.4686_tc82_avglc23466_sdlc7887,2024-06-10 19:46:14,2024-06-11 01:19:02.064147,0.436051,27743.448481,2024-06-11 03:28:37.448480772,19968.064147,30.332599,0.805960,2024-06-11 01:19:02.064147000,False,82,5.4686,66.560214,0.042144,0.030333,0.000456
155001,151,2024-06-15,r5.4686_tc82_avglc23466_sdlc7887,2024-06-15 19:37:48,2024-06-16 01:31:57.258927,0.301468,35008.340124,2024-06-16 05:21:16.340123883,21249.258927,32.278805,0.695112,2024-06-16 01:31:57.258927000,True,82,5.4686,70.830863,0.053180,0.032279,0.000456
155002,152,2024-06-18,r5.4686_tc82_avglc23466_sdlc7887,2024-06-18 19:39:27,2024-06-18 22:58:48.801763,0.386686,30408.212138,2024-06-19 04:06:15.212137841,11961.801763,18.170641,0.608279,2024-06-18 22:58:48.801763000,True,82,5.4686,39.872673,0.046192,0.018171,0.000456
155003,153,2024-06-21,r5.4686_tc82_avglc23466_sdlc7887,2024-06-21 20:58:40,2024-06-22 01:41:22.560361,0.278963,36223.200017,2024-06-22 07:02:23.200017253,16962.560361,25.767072,0.593195,2024-06-22 01:41:22.560361000,True,82,5.4686,56.541868,0.055025,0.025767,0.000456
155004,154,2024-06-28,r5.4686_tc82_avglc23466_sdlc7887,2024-06-28 17:59:01,2024-06-29 00:25:29.456401,0.537918,22244.542703,2024-06-29 00:09:45.542702599,23188.456401,33.790696,0.950000,2024-06-29 00:09:45.542702599,False,82,5.4686,77.294855,0.033791,0.033791,0.000456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162995,158,2024-07-09,r8.294_tc54_avglc29435_sdlc7319,2024-07-09 19:22:10,2024-07-10 03:34:47.452378,0.298409,15272.392642,2024-07-09 23:36:42.392642008,29557.452378,35.185896,0.950000,2024-07-09 23:36:42.392642008,False,54,8.2940,98.524841,0.035186,0.035186,0.000691
162996,159,2024-07-11,r8.294_tc54_avglc29435_sdlc7319,2024-07-11 18:18:19,2024-07-12 01:14:42.762356,0.256707,16249.840553,2024-07-11 22:49:08.840553043,24983.762356,37.437827,0.950000,2024-07-11 22:49:08.840553043,False,54,8.2940,83.279208,0.037438,0.037438,0.000691
162997,160,2024-07-17,r8.294_tc54_avglc29435_sdlc7319,2024-07-17 18:02:21,2024-07-17 23:58:01.831828,0.500006,10547.244508,2024-07-17 20:58:08.244508348,21340.831828,24.299679,0.950000,2024-07-17 20:58:08.244508348,False,54,8.2940,71.136106,0.024300,0.024300,0.000691
162998,161,2024-07-19,r8.294_tc54_avglc29435_sdlc7319,2024-07-19 20:27:49,2024-07-20 07:03:49.757888,0.400715,12874.490448,2024-07-20 00:02:23.490447644,38160.757888,29.661395,0.950000,2024-07-20 00:02:23.490447644,False,54,8.2940,127.202526,0.029661,0.029661,0.000691


# Synthetic User Data

In [17]:
# regions = [
# 'CAISO_NORTH',
# 'SPP_TX',
# 'ERCOT_EASTTX',
# 'FPL',
# 'SOCO',
# 'PJM_CHICAGO',
# 'LDWP',
# 'PJM_DC',
# 'NYISO_NYC'
# ]
regions = [
    "PJM_CHICAGO",
]
region = regions[0]

synth_data = generated_data.copy(deep=True)
synth_data["session_start_time"] = pd.to_datetime(synth_data["plug_in_time"])
synth_data["unplug_time"] = pd.to_datetime(synth_data["unplug_time"])

import pickle

actual_pickle = s3.load_file(file="pjm_actual.pkl")
HISTORICAL_ACTUAL_CACHE = pickle.loads(actual_pickle)

forecast_pickle = s3.load_file(file="pjm_forecast.pkl")
HISTORICAL_FORECAST_CACHE = pickle.loads(actual_pickle)

# Cached version of the get_*_data functions

In [21]:
%%time


def precache_actual_data(synth_data, regions):
    distinct_dates = [
        datetime.strptime(date, "%Y-%m-%d").date()
        for date in synth_data["distinct_dates"].unique().tolist()
    ]
    all_dates_regions = [
        (date, region) for date in distinct_dates for region in regions
    ]

    def get_actual_data_for_region_date(date, region):
        start = pd.to_datetime(date)
        end = start + pd.Timedelta("2d")
        return (
            region,
            date,
            actual_data.get_historical_pandas(
                start - pd.Timedelta("9h"),
                end + pd.Timedelta("9h"),
                region,
            ),
        )

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        result = executor.map(
            get_actual_data_for_region_date,
            [date for (date, region) in all_dates_regions],
            [region for (date, region) in all_dates_regions],
        )
    result = list(result)

    return {(region, date): data for (region, date, data) in result}


HISTORICAL_ACTUAL_CACHE = precache_actual_data(synth_data, regions)

CPU times: user 2.36 s, sys: 113 ms, total: 2.47 s
Wall time: 3.25 s


In [22]:
%%time


def precache_fcst_data(synth_data, regions):
    distinct_dates = [
        datetime.strptime(date, "%Y-%m-%d").date()
        for date in synth_data["distinct_dates"].unique().tolist()
    ]
    all_dates_regions = [
        (date, region) for date in distinct_dates for region in regions
    ]

    def get_fsct_data_for_region_date(date, region):
        start = pd.to_datetime(date)
        end = pd.to_datetime(date) + pd.Timedelta("1d")
        return (
            region,
            date,
            hist_data.get_historical_forecast_pandas(
                start - pd.Timedelta("9h"),
                end + pd.Timedelta("9h"),
                region,
            ),
        )

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        result = executor.map(
            get_fsct_data_for_region_date,
            [date for (date, region) in all_dates_regions],
            [region for (date, region) in all_dates_regions],
        )
    result = list(result)
    return {(region, date): data for (region, date, data) in result}


HISTORICAL_FORECAST_CACHE = precache_fcst_data(synth_data, regions)


 Warnings Returned: {'region': 'PJM_CHICAGO', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-01-20 15:00:00+0000', tz='UTC'), 'end': Timestamp('2023-01-21 14:55:00+0000', tz='UTC')} {'data_point_period_seconds': 300, 'region': 'PJM_CHICAGO', 'signal_type': 'co2_moer', 'units': 'lbs_co2_per_mwh', 'warnings': [{'type': 'MISSING_DATA', 'message': 'You made a valid request, but we have some datapoints missing. This can happen for a variety of reasons. If the data is less than a week old, we will most likely fill in this data soon. If it is older than that, we are missing enough important grid data during this time period that we are not able to generate an accurate estimate'}], 'model': {'date': '2023-03-01'}, 'generated_at_period_seconds': 300}

 Warnings Returned: {'region': 'PJM_CHICAGO', 'signal_type': 'co2_moer', 'horizon_hours': 24, 'start': Timestamp('2023-06-01 15:00:00+0000', tz='UTC'), 'end': Timestamp('2023-06-02 14:55:00+0000', tz='UTC')} {'data_point

Exception: 
API Response Error: 401, <html>
<head><title>401 Authorization Required</title></head>
<body>
<center><h1>401 Authorization Required</h1></center>
<hr><center>nginx</center>
</body>
</html>
 [2c31e74027dd0d613dec9f7ed1250b3c]

---

In [16]:
HISTORICAL_FORECAST_CACHE

{('PJM_CHICAGO',
  datetime.date(2023, 1, 2)):                    point_time   value
 0   2023-01-01 15:00:00+00:00  1215.0
 1   2023-01-01 15:05:00+00:00  1223.0
 2   2023-01-01 15:10:00+00:00  1223.0
 3   2023-01-01 15:15:00+00:00  1223.0
 4   2023-01-01 15:20:00+00:00  1222.0
 ..                        ...     ...
 788 2023-01-04 08:40:00+00:00  1179.0
 789 2023-01-04 08:45:00+00:00  1177.0
 790 2023-01-04 08:50:00+00:00  1175.0
 791 2023-01-04 08:55:00+00:00  1175.0
 792 2023-01-04 09:00:00+00:00  1175.0
 
 [793 rows x 2 columns],
 ('PJM_CHICAGO',
  datetime.date(2023, 1, 6)):                    point_time   value
 0   2023-01-05 15:00:00+00:00  1221.0
 1   2023-01-05 15:05:00+00:00  1217.0
 2   2023-01-05 15:10:00+00:00  1219.0
 3   2023-01-05 15:15:00+00:00  1224.0
 4   2023-01-05 15:20:00+00:00  1224.0
 ..                        ...     ...
 788 2023-01-08 08:40:00+00:00  1187.0
 789 2023-01-08 08:45:00+00:00  1187.0
 790 2023-01-08 08:50:00+00:00  1187.0
 791 2023-01-08 08:55:0

In [13]:
def get_historical_fcst_data_cached(session_start_time, horizon, region):
    time_zone = efu.get_timezone_from_dict(region)
    session_start_time_utc = pd.Timestamp(
        efu.convert_to_utc(session_start_time, time_zone)
    )
    date = session_start_time.date()
    if (region, date) not in HISTORICAL_FORECAST_CACHE.keys():
        print(type(date), date)
        start = pd.to_datetime(date)
        end = pd.to_datetime(date) + pd.Timedelta("1d")
        HISTORICAL_FORECAST_CACHE[(region, date)] = (
            hist_data.get_historical_forecast_pandas(
                start - pd.Timedelta("9h"),
                end + pd.Timedelta("9h"),
                region,
            )
        )
    cache = HISTORICAL_FORECAST_CACHE[(region, date)]

    # make this match efu.get_historical_fsct_data
    generated_at_times = cache["generated_at"].unique()
    generated_at = max([t for t in generated_at_times if t < session_start_time_utc])
    df = cache[cache["generated_at"] == generated_at].copy()
    return df.iloc[: math.ceil(horizon / 12) * 12]


def get_historical_actual_data_cached(session_start_time, horizon, region):
    time_zone = efu.get_timezone_from_dict(region)
    session_start_time_utc = pd.Timestamp(
        efu.convert_to_utc(session_start_time, time_zone)
    )
    date = session_start_time.date()

    if (region, date) not in HISTORICAL_ACTUAL_CACHE.keys():
        start = pd.to_datetime(date)
        end = pd.to_datetime(date) + pd.Timedelta("2d")
        HISTORICAL_ACTUAL_CACHE[(region, date)] = actual_data.get_historical_pandas(
            start - pd.Timedelta("9h"),
            end + pd.Timedelta("9h"),
            region,
        )
    cache = HISTORICAL_ACTUAL_CACHE[(region, date)]

    t_start = max(
        [t for t in cache["point_time"].unique() if t < session_start_timet_time_utc]
    )
    df = cache[cache["point_time"] >= t_start].copy()
    return df.iloc[: math.ceil(horizon / 12) * 12 + 1].reset_index(drop=True)

## API Data with caching

In [14]:
%%time

synth_data["moer_data"] = synth_data.apply(
    lambda x: get_historical_fcst_data_cached(
        x.session_start_time, math.ceil(x.total_intervals_plugged_in), region=region
    ),
    axis=1,
)

KeyError: 'generated_at'

In [ ]:

%%time

synth_data['moer_data_actual'] = synth_data.apply(
    lambda x: get_historical_actual_data_cached(
    x.session_start_time,
    math.ceil(x.total_intervals_plugged_in),
    region = region
    ), axis = 1
)

## MOER - No Optimization - Actual Emissions

In [ ]:
%%time

get_charging_schedule_lambda = lambda x: x["usage"].values.flatten()
get_total_emissions_lambda = lambda x: x["emissions_co2e_lb"].sum()

synth_data["charger_baseline_actual_api"] = synth_data.apply(
    lambda x: efu.get_schedule_and_cost_api(
        x.power_output_rate,
        math.ceil(min(x.total_seconds_to_95, x.length_of_session_in_seconds) / 300.0)
        * 5.0,
        x.moer_data_actual,
        optimization_method="baseline",
    ),
    axis=1,
)

synth_data["baseline_charging_schedule_api"] = synth_data[
    "charger_baseline_actual_api"
].apply(get_charging_schedule_lambda)
synth_data["baseline_actual_emissions_api"] = synth_data[
    "charger_baseline_actual_api"
].apply(get_total_emissions_lambda)

## MOER - Simple Optimization - Forecast Emissions

In [ ]:
%%time


# TODO: I feel like this slicing might lead to bugs in the future
def get_total_emission(moer, schedule):
    x = np.array(schedule).flatten()
    return np.dot(moer[: x.shape[0]], x)


synth_data["charger_simple_forecast"] = synth_data.apply(
    lambda x: efu.get_schedule_and_cost_api(
        x.power_output_rate,
        int(
            math.ceil(min(x.total_seconds_to_95, x.get_schedule_and_cost_api) / 300.0)
            * 5
        ),
        x.moer_data,
        optimization_method="simple",
    ),
    axis=1,
)

synth_data["simple_charging_schedule"] = synth_data["charger_simple_forecast"].apply(
    get_charging_schedule_lambda
)
synth_data["simple_estimated_emissions"] = synth_data["charger_simple_forecast"].apply(
    get_total_emissions_lambda
)
synth_data["simple_actual_emissions"] = synth_data.apply(
    lambda x: get_total_emission(
        x.moer_data_actual["value"],
        x.charger_simple_forecast.energy_usage_mwh,
    ),
    axis=1,
)

In [ ]:
%%time

synth_data["charger_simple_actual"] = synth_data.apply(
    lambda x: efu.get_schedule_and_cost_api(
        x.power_output_rate,
        int(
            math.ceil(
                min(x.total_seconds_to_95, x.length_of_session_in_seconds) / 300.0
            )
            * 5
        ),
        x.moer_data_actual,
        optimization_method="simple",
    ),
    axis=1,
)


synth_data["simple_actual_charging_schedule"] = synth_data[
    "charger_simple_actual"
].apply(get_charging_schedule_lambda)
synth_data["simple_ideal_emissions"] = synth_data["charger_simple_actual"].apply(
    get_total_emissions_lambda
)

---

# store the results

In [ ]:
from optimizer.s3 import s3_utils

s3 = s3_utils()
cols = [
    "user_type",
    "power_output_rate",
    "distinct_dates",
    "session_start_time",
    "total_intervals_plugged_in",
    "charged_kWh_actual",
    "MWh_fraction",
    "simple_actual_emissions",
    "baseline_actual_emissions",
    "simple_estimated_emissions",
    "simple_ideal_emissions",
]

s3.store_csvdataframe(
    synth_data[cols], f"results_v2/20240726_1k_synth_users_163_days_{region}.csv"
)